In [1]:
# ensure cwd is project root
import os
root = r"C:\Users\Dell\Documents\Customer-Churn-Prediction"
if os.getcwd() != root:
    os.chdir(root)
print("cwd:", os.getcwd())


cwd: C:\Users\Dell\Documents\Customer-Churn-Prediction
